<a href="https://colab.research.google.com/github/jekwa1/Project/blob/main/GroupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#mount to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load cases_malaysia csv file using Pandas
folder = '/content/drive/My Drive/HIAProject302/'
filename_1 = folder+'cases_malaysia.csv'
case_my= pd.read_csv(filename_1)

#load deaths_malaysia csv file using Pandas
folder = '/content/drive/My Drive/HIAProject302/'
filename_2 = folder+'deaths_malaysia.csv'
death_my= pd.read_csv(filename_2)

#load icu csv file using Pandas
folder = '/content/drive/My Drive/HIAProject302/'
filename_3 = folder+'icu.csv'
icu_my= pd.read_csv(filename_3)


Verifying Datasets (case_malaysia.csv), (death_malaysia.csv) and (icu.csv)

In [ ]:
#Check null in case_my
case_my.isnull().sum()

#Check null in death_my
death_my.isnull().sum()

#Check null in icu_my
icu_my.isnull().sum()

Selecting required variables from each datasets

In [ ]:
#Select interested column in case_my
case_my_new =case_my.loc[:,['date','cases_new','cases_0_4', 'cases_5_11', 'cases_12_17',
'cases_18_29', 'cases_30_39', 'cases_40_49', 'cases_50_59',
'cases_60_69', 'cases_70_79', 'cases_80']]
print(case_my_new)
print(case_my_new.columns)
#check null value
case_my_new.isnull().sum()



In [ ]:
#Select interested column in death_my
death_my_new = death_my.loc[:,['date','deaths_new']]
print(death_my_new)
print(death_my_new.columns)
#check null value
death_my_new.isnull().sum()



In [ ]:
#Select interested column in icu_my
icu_my_new = icu_my.loc[:,['date','icu_covid']]
icu_my_new_group = icu_my_new.groupby('date')['icu_covid'].sum().reset_index()
print(icu_my_new_group)
#check null value
icu_my_new_group.isnull().sum()

Creating new dataframe (my_covid) - Merging datasets (based on date)

In [ ]:
# Ensuring formate of date - datetime type
case_my_new['date'] = pd.to_datetime(case_my_new['date'])
death_my_new['date'] = pd.to_datetime(death_my_new['date'])
icu_my_new_group['date'] = pd.to_datetime(icu_my_new_group['date'])

# Merge DataFrames
my_covid = pd.merge(case_my_new, death_my_new, on='date', how='outer')
my_covid = pd.merge(my_covid, icu_my_new_group, on='date', how='outer')

# Sort DataFrame by 'date'
my_covid = my_covid.sort_values(by='date')

# Fill NaN values with 0
my_covid = my_covid.fillna(0)



Validating new data frame (my_covid)

In [ ]:
## Validation
print(my_covid)
print(my_covid.isnull().sum())

num_rows, num_columns = my_covid.shape
print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

# Ensuring formate of date - datetime type
print(my_covid.dtypes)
# my_covid['date'] = pd.to_datetime(my_covid['date'])

my_covid.to_csv('my_covid.csv', index=False)

Creating new dataframe based on 7 days aggregate (sum) and filtered for date (01/01/2020 - 30/06/2023) = my_covid_group

In [ ]:
# Filtering based on date (01/01/2000 - 30/06/2023)
start_date = '2000-01-01'
end_date = '2023-06-30'

date_mask = (my_covid['date'] >= start_date) & (my_covid['date'] <= end_date)
filtered_my_covid = my_covid[date_mask].copy()  # Make a copy to avoid SettingWithCopyWarning

# Save the filtered dataframe with 'date' column
filtered_my_covid.to_csv('filtered_my_covid.csv', index=False)

# New dataframe based on 7 days aggregate (sum) = my_covid_group

filtered_my_covid['date'] = pd.to_datetime(filtered_my_covid['date'])
filtered_my_covid.set_index('date', inplace=True)

my_covid_group = filtered_my_covid.resample('7D').sum()
my_covid_group.reset_index(inplace=True)
print(my_covid_group)

my_covid_group.to_csv('my_covid_group.csv', index=False)

Create new dataframe based on 7-days average (mean)= seven_days_mean and validate

In [ ]:
seven_days_mean = filtered_my_covid.resample('7D').mean()
seven_days_mean = seven_days_mean.round(2)
seven_days_mean.reset_index(inplace=True)

## Validation
print(seven_days_mean)
print(seven_days_mean.isnull().sum())

seven_days_mean.to_csv('seven_days_mean.csv', index=False)
print(seven_days_mean.columns)

Creating working data frame from seven_days_mean and factoring in predominant circulating COVID-19 variant

In [ ]:
# New dataframe:seven_days_mean_variant (this with predominant circulating variant)

seven_days_mean_variant = seven_days_mean.copy()

# Create a new column 'prominent_variant'
seven_days_mean_variant['prominent_variant'] = pd.cut(
    seven_days_mean_variant['date'],
    bins=[pd.to_datetime('2000-01-01'), pd.to_datetime('2021-06-08'), pd.to_datetime('2021-07-05'), pd.to_datetime('2021-12-20'), pd.to_datetime('2100-01-01')],
    labels=['other', 'beta', 'delta', 'omicron'],
    right=False
)

## Validation
print(seven_days_mean_variant)
print(seven_days_mean_variant.isnull().sum())

print(seven_days_mean_variant.head())
seven_days_mean_variant.to_csv('seven_days_mean_variant.csv', index=False)

Statistical Analysis

In [ ]:
#Create a pie chart for prominent variant
seven_days_mean_variant['prominent_variant'].value_counts().plot(kind='pie',autopct="%.2f%%")
